In [ ]:
import matplotlib.pyplot as plt
import os
import re
import string
import tensorflow as tf
import pandas as pd

from utils import predicted_test_data_to_result_csv
from keras import layers, losses, Input, Model
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Activation, \
    Flatten, LSTM, SpatialDropout1D, Bidirectional, MultiHeadAttention, LayerNormalization, Lambda, \
    GlobalAveragePooling1D, Dropout
from keras.losses import sparse_categorical_crossentropy
from keras.metrics import sparse_categorical_accuracy
from keras.optimizers import Adam, SGD
from keras.models import Sequential

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
train_path = "data/base/goodreads_train.csv"
result_path = "data/base/goodreads_test.csv"
frac_ratio = 0.2

In [ ]:
max_features = 20000  # Maximum vocab size.
sequence_length = 200

In [ ]:
df = pd.read_csv(train_path, sep=",")

In [ ]:
index = df[(df['rating'] == 0)].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
x_train = df.sample(frac=frac_ratio)
x_val = df.drop(x_train.index)

In [ ]:
y_train = x_train.pop('rating')
y_train = y_train - 1

y_val = x_val.pop('rating')
y_val = y_val - 1

In [ ]:
x_train = x_train["review_text"]
x_val = x_val["review_text"]

In [ ]:
raw_train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10, reshuffle_each_iteration=False)
raw_val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(10, reshuffle_each_iteration=False)

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_spoilers = tf.strings.regex_replace(lowercase, '\*\* spoiler alert \*\*', ' ')
    return tf.strings.regex_replace(stripped_spoilers,
                                    '[%s]' % re.escape(string.punctuation),
                                    '')

In [ ]:
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
epochs = 10
model_nb = 1

embedding_dim = 32 # Embedding size for each token
num_heads = 2 # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
learning_rate = 0.01
batch_size = 64
dropout_rate = 0.1

In [ ]:
raw_train_dataset = raw_train_dataset.batch(batch_size=batch_size)
raw_val_dataset = raw_val_dataset.batch(batch_size=batch_size)

In [ ]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_dataset.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

val_text = raw_val_dataset.map(lambda x, y: x)
vectorize_layer.adapt(val_text)

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [ ]:
train_ds = raw_train_dataset.map(vectorize_text)
val_ds = raw_val_dataset.map(vectorize_text)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

input_text = Input(shape=(sequence_length,))

embedding_layer = Embedding(max_features + 1, embedding_dim, input_length=sequence_length)(input_text)

x = embedding_layer
for _ in range(6):
    old = x
    x = LayerNormalization()(x)
    x = MultiHeadAttention(num_heads, embedding_dim)(x, x)
    x = x + old
    old = x
    x = LayerNormalization()(x)
    x = Dense(embedding_dim * 2, activation="relu")(x)
    x = Dense(embedding_dim, activation="relu")(x)
    x = x + old

mlp_head_input = Lambda(lambda x: x[:, 0])(x)

dense = Dense(512, activation='relu')(mlp_head_input)

output = Dense(5, activation='softmax')(dense)

transformer_model = Model(input_text, output)

transformer_model.summary()

In [ ]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, ff_dim, rate=0):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads,
                                      key_dim=embedding_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation='relu'),
             Dense(embedding_dim), ]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)

        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embedding_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)

        return x + positions

In [ ]:
inputs = Input(shape=(sequence_length,))
embedding_layer = TokenAndPositionEmbedding(maxlen=sequence_length, vocab_size=max_features, embedding_dim=embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim=embedding_dim, num_heads=num_heads, ff_dim=ff_dim)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(dropout_rate)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(dropout_rate)(x)
outputs = Dense(5, activation='softmax')(x)

transformer_model = Model(inputs=inputs, outputs=outputs)

In [ ]:
transformer_model.compile(loss=sparse_categorical_crossentropy,
                          optimizer=Adam(learning_rate=learning_rate),
                          metrics=sparse_categorical_accuracy)

In [ ]:
exp_name = f'transformer_model_{model_nb}_num_heads{num_heads}_emb_dim_{embedding_dim}_ff_dim_{ff_dim}_lr_{learning_rate}_bs_{batch_size}_dr_{dropout_rate}'

In [ ]:
transformer_model.fit(train_ds,
                      validation_data=val_ds,
                      epochs=epochs,
                      callbacks=[
                          tf.keras.callbacks.TensorBoard("logs/transformer/" + exp_name),
                          tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
                                 ])

In [ ]:
df_test = pd.read_csv(result_path, sep=",")

df_test_modified = df_test.drop(columns=[
    'user_id',
    'book_id',
    'review_id',
    'date_added',
    'date_updated',
    'read_at',
    'started_at',
    'n_votes',
    'n_comments'
], inplace=False)

In [ ]:
export_model = tf.keras.Sequential([
    vectorize_layer,
    transformer_model
])

In [ ]:
predicted_test_data = export_model.predict(df_test_modified)

In [ ]:
df_test.head()

In [ ]:
predicted_test_data_to_result_csv(df_test, predicted_test_data, exp_name)